In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import string,re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import operator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras import utils as np_utils
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.layers import Activation,Conv2D,Flatten,Dropout,Dense,Conv1D,LSTM,GRU,MaxPool1D,MaxPooling1D
from tensorflow.python.client import device_lib 

from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.layers.embeddings import Embedding
from sklearn.manifold import TSNE

In [18]:
from nltk.stem import PorterStemmer

In [14]:
df=pd.read_csv('train.csv')
df.head()

,Review Text,Review Title,topic
0,"Did nothing for me, didn't help lost even with...",Useless,Shipment and delivery
1,"Did nothing for me, didn't help lost even with...",Useless,Not Effective
2,I have bought these bags and immediately open...,TRASH!!! Do not buy these bags it’s a waist of...,Customer Service
3,Gave me an allergic reaction on my face :(,Do not recommend,Allergic
4,These don't compare to the name brand wipes. F...,Can't tackle big messes,Texture


In [6]:

review_title=df['Review Title']
review_text=df['Review Text']
topic=df.topic
review=review_title+' '+review_text

In [19]:
sw=stopwords.words('english')
ps=PorterStemmer()

In [25]:
def cleatext(text):
    
    text=text.translate(text.maketrans("","",string.punctuation))
    text=[ps.stem(w) for w in text.split(' ') if w not in sw]
    text=' '.join(text)
    return text

In [27]:
review=review.apply(cleatext)

In [29]:
tk=Tokenizer()

tk.fit_on_texts(review)


encoded_docs=tk.texts_to_sequences(review)

vocab_size=len(tk.word_index)+1
max_length=max([len(s.split(' ')) for s in review])
padded_docs=pad_sequences(encoded_docs,maxlen=max_length,padding='post')

In [45]:
lb=LabelEncoder()
onh=OneHotEncoder()
lb.fit(topic)

topic_encoded=lb.transform(topic)
y=np_utils.to_categorical(topic_encoded,num_classes=21)

In [53]:
X_train,X_test,y_train,y_test=train_test_split(padded_docs,y,test_size=0.2)

In [84]:


model=Sequential()
model.add(Embedding(vocab_size,output_dim=200,input_length=max_length))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))

model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(32,activation='relu'))
model.add(Dense(21,activation='softmax'))
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [85]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 315, 200)          1586200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 315, 200)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 311, 64)           64064     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 77, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_9 (Dense)              (None, 21)                693       
Total para

In [86]:

with tf.device('/gpu:0'):
    model.fit(X_train,y_train,epochs=100,batch_size=20,validation_data=(X_test,y_test))
    

Train on 4767 samples, validate on 1192 samples
Epoch 1/100
4767/4767 [==============================] - 52s 11ms/step - loss: 2.6287 - acc: 0.1991 - val_loss: 2.5982 - val_acc: 0.2030
Epoch 2/100
4767/4767 [==============================] - 49s 10ms/step - loss: 2.5932 - acc: 0.1997 - val_loss: 2.6044 - val_acc: 0.2030
Epoch 3/100
4767/4767 [==============================] - 48s 10ms/step - loss: 2.5867 - acc: 0.1997 - val_loss: 2.6079 - val_acc: 0.2030
Epoch 4/100
4767/4767 [==============================] - 48s 10ms/step - loss: 2.5872 - acc: 0.1997 - val_loss: 2.6021 - val_acc: 0.2030
Epoch 5/100
4767/4767 [==============================] - 48s 10ms/step - loss: 2.5883 - acc: 0.1997 - val_loss: 2.6128 - val_acc: 0.2030
Epoch 6/100
4767/4767 [==============================] - 49s 10ms/step - loss: 2.5879 - acc: 0.1997 - val_loss: 2.6003 - val_acc: 0.2030
Epoch 7/100
4767/4767 [==============================] - 48s 10ms/step - loss: 2.5880 - acc: 0.1997 - val_loss: 2.5983 - val_acc: 

4767/4767 [==============================] - 51s 11ms/step - loss: 1.5790 - acc: 0.5219 - val_loss: 2.3628 - val_acc: 0.3960
Epoch 61/100
4767/4767 [==============================] - 51s 11ms/step - loss: 1.5554 - acc: 0.5236 - val_loss: 2.3766 - val_acc: 0.3733
Epoch 62/100
4767/4767 [==============================] - 51s 11ms/step - loss: 1.5061 - acc: 0.5368 - val_loss: 2.4524 - val_acc: 0.3918
Epoch 63/100
4767/4767 [==============================] - 51s 11ms/step - loss: 1.4711 - acc: 0.5439 - val_loss: 2.3938 - val_acc: 0.3859
Epoch 64/100
4767/4767 [==============================] - 50s 11ms/step - loss: 1.4361 - acc: 0.5505 - val_loss: 2.3972 - val_acc: 0.4060
Epoch 65/100
4767/4767 [==============================] - 48s 10ms/step - loss: 1.4001 - acc: 0.5647 - val_loss: 2.4325 - val_acc: 0.4044
Epoch 66/100
4767/4767 [==============================] - 48s 10ms/step - loss: 1.3963 - acc: 0.5624 - val_loss: 2.4254 - val_acc: 0.4010
Epoch 67/100
4767/4767 [=======================

In [88]:
model.evaluate(X_test,y_test)

1192/1192 [==============================] - 5s 4ms/step


[3.5351362292398543, 0.4152684563758389]

In [89]:
dt=pd.read_csv('test.csv')
dt.head()

,Review Text,Review Title
0,I use chia seed in my protein shakes. These ta...,Bad tast
1,I use chia seed in my protein shakes. These ta...,Bad tast
2,Don’t waste your money.,No change. No results.
3,I use the book 'Fortify Your Life' by Tieraona...,"Good Vegan Choice, Poor Non Vegan Choice"
4,I use the book 'Fortify Your Life' by Tieraona...,"Good Vegan Choice, Poor Non Vegan Choice"


In [74]:

dt_review_title=dt['Review Title']
dt_review_text=dt['Review Text']

dt_review=dt_review_title+' '+dt_review_text

In [90]:
dt_review=dt_review.apply(cleatext)

In [91]:
dt_encoded_docs=tk.texts_to_sequences(dt_review)


dt_padded_docs=pad_sequences(dt_encoded_docs,maxlen=max_length,padding='post')

In [92]:
dt_padded_docs

array([[  10,    2,    1, ...,    0,    0,    0],
       [  10,    2,    1, ...,    0,    0,    0],
       [ 110,  104,  110, ...,    0,    0,    0],
       ...,
       [  62,   73,  160, ...,    0,    0,    0],
       [3309,   10, 1239, ...,    0,    0,    0],
       [ 243,  349,   21, ...,    0,    0,    0]])

In [93]:
y_pred=model.predict_classes(dt_padded_docs)

In [94]:
y_pred=lb.inverse_transform(y_pred)

In [95]:
len(set(y_pred))

20

In [96]:

tp=pd.DataFrame({'Review Text':dt_review_text,'Review Title':dt_review_title,'topic':y_pred})
tp.to_csv('Submission_1_LSTM.csv',index=False)